In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Visual import animated_frames, save_frames
from Simulation import run
import numpy as np
from tqdm.autonotebook import tqdm

In [3]:
l = 1
d = l*2e-3
epsilon = d*1e-4

In [4]:
N = 50 #number of vertexes
Frames = 10 #number of frames between adding vercites
M = 50 #number of segment adding iteration between adding vertices
M = int(M/Frames)

In [5]:
np.random.seed(3)
animation_segments,animation_vertices = run(N,M,Frames,l,d,epsilon)

In [6]:
%matplotlib widget
animated_frames(animation_segments, animation_vertices)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=444)

interactive(children=(Play(value=0, description='Press play', interval=200, max=444), Output()), _dom_classes=…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [7]:
save_frames(animation_segments, animation_vertices,"results_multi_vertices_3",3)

NameError: name 'animation_segments' is not defined

In [ ]:
!./nice_gif.sh results_multi_vertices_1/%06d.png animations/anim3.mp4 animations/anim3.gif

In [27]:
from Simulation import reset, find_segments, segments_adding
segments, active_vertices, animation_vertices, animation_segments = reset()
active_vertices.append(np.array([1,1]))
active_vertices.append(np.array([1,-1]))
active_vertices.append(np.array([0.5,0.5]))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
for f in tqdm(range(100)):
    L = len(segments)
    active_segments, segments_vertices = segments_adding(10,active_vertices,active_segments,segments_vertices,segments,2e-3,2e-7)
    if L != len(segments):
        animation_vertices.append(np.array(active_vertices))
        animation_segments.append(np.array(segments))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
animation_vertices.append(np.array(active_vertices))
animation_segments.append(np.array(segments))
segments = np.vstack(np.array(segments)).T
%matplotlib widget
animated_frames(animation_segments, animation_vertices)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=90)

interactive(children=(Play(value=0, description='Press play', interval=200, max=90), Output()), _dom_classes=(…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [30]:
###TEST 6
from Simulation import reset, find_segments, segments_adding
segments, active_vertices, animation_vertices, animation_segments = reset()
active_vertices.append(np.array([1,1]))
active_vertices.append(np.array([1,-1]))
active_vertices.append(np.array([0.5,0.5]))
active_vertices.append(np.array([0.5,-0.5]))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
for f in range(150):
    L = len(segments)
    active_segments, segments_vertices = segments_adding(10,active_vertices,active_segments,segments_vertices,segments,2e-3,2e-7)
    if L != len(segments):
        animation_vertices.append(np.array(active_vertices))
        animation_segments.append(np.array(segments))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
animation_vertices.append(np.array(active_vertices))
animation_segments.append(np.array(segments))
segments = np.vstack(np.array(segments)).T
%matplotlib widget
animated_frames(animation_segments, animation_vertices)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=82)

interactive(children=(Play(value=0, description='Press play', interval=200, max=82), Output()), _dom_classes=(…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [31]:
###TEST 7
from Simulation import reset, find_segments, segments_adding
segments, active_vertices, animation_vertices, animation_segments = reset()
active_vertices.append(np.array([0,1]))
segments.append((2e-3,0))
segments.append((2*2e-3,0))
segments.append((3*2e-3,0))
segments.append((4*2e-3,0))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
for f in range(50):
    L = len(segments)
    active_segments, segments_vertices = segments_adding(10,active_vertices,active_segments,segments_vertices,segments,2e-3,2e-7)
    if L != len(segments):
        animation_vertices.append(np.array(active_vertices))
        animation_segments.append(np.array(segments))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
animation_vertices.append(np.array(active_vertices))
animation_segments.append(np.array(segments))
segments = np.vstack(np.array(segments)).T
%matplotlib widget
animated_frames(animation_segments, animation_vertices)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=50)

interactive(children=(Play(value=0, description='Press play', interval=200, max=50), Output()), _dom_classes=(…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [32]:
###TEST 8
from Simulation import reset, find_segments, segments_adding
segments, active_vertices, animation_vertices, animation_segments = reset()
active_vertices.append(np.array([0.5,0]))
segments.append((1,0))
segments.append((0.5,0.5))
segments.append((0.5,-0.5))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
for f in range(150):
    L = len(segments)
    active_segments, segments_vertices = segments_adding(10,active_vertices,active_segments,segments_vertices,segments,2e-3,2e-7)
    if L != len(segments):
        animation_vertices.append(np.array(active_vertices))
        animation_segments.append(np.array(segments))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
animation_vertices.append(np.array(active_vertices))
animation_segments.append(np.array(segments))
print(len(segments))
segments = np.vstack(np.array(segments)).T
%matplotlib widget
animated_frames(animation_segments, animation_vertices)

1000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=25)

interactive(children=(Play(value=0, description='Press play', interval=200, max=25), Output()), _dom_classes=(…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [35]:
from Simulation import reset, find_segments, segments_adding
segments, active_vertices, animation_vertices, animation_segments, active_segments, segments_vertices = reset()
active_vertices.append(np.array([1,1]))
active_vertices.append(np.array([1,-1]))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
for f in range(102):
    L = len(segments)
    active_segments, segments_vertices = segments_adding(10,active_vertices,active_segments,segments_vertices,segments,2e-3,2e-7)
    if L != len(segments):
        animation_vertices.append(np.array(active_vertices))
        animation_segments.append(np.array(segments))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
animation_vertices.append(np.array(active_vertices))
animation_segments.append(np.array(segments))
segments = np.vstack(np.array(segments)).T
%matplotlib widget
animated_frames(animation_segments, animation_vertices)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=101)

interactive(children=(Play(value=0, description='Press play', interval=200, max=101), Output()), _dom_classes=…

<function Visual.animated_frames.<locals>.update(f: int = 0)>